In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import datetime as dt

import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)

## 0. Read data, info

In [26]:
data_file_path = "C:/Users/Bas/OneDrive/MSc. Artificial Intelligence VU/MSc. AI Year 1/Data Mining Techniques/Assignment 2/data/"
train = pd.read_csv(data_file_path + "training_set_VU_DM.csv", sep=',')
print(len(train)) #4,958,347 rows
print(np.sort(train['srch_id'].unique())[:10])
train.head(5)

4958347
[ 1  4  6  8 11 12 17 21 25 28]


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [27]:
test = pd.read_csv(data_file_path + "test_set_VU_DM.csv", sep=',')
print(len(test)) #4,959,183 rows
print(np.sort(test['srch_id'].unique())[:10])
# test.head(5)

# missing columns in test:             position, click_bool, gross_bookings_usd, nor booking_bool

4959183
[ 1  3  6  7 10 11 12 13 14 16]


In [5]:
#export 5 training examples to excel to easily view
for i in np.sort(train['srch_id'].unique())[:5]:
    train[train['srch_id'] == i].to_excel('srch_id_' + str(i) + '.xlsx', sheet_name='new_sheet')

In [6]:
train.columns.to_list()

['srch_id',
 'date_time',
 'site_id',
 'visitor_location_country_id',
 'visitor_hist_starrating',
 'visitor_hist_adr_usd',
 'prop_country_id',
 'prop_id',
 'prop_starrating',
 'prop_review_score',
 'prop_brand_bool',
 'prop_location_score1',
 'prop_location_score2',
 'prop_log_historical_price',
 'position',
 'price_usd',
 'promotion_flag',
 'srch_destination_id',
 'srch_length_of_stay',
 'srch_booking_window',
 'srch_adults_count',
 'srch_children_count',
 'srch_room_count',
 'srch_saturday_night_bool',
 'srch_query_affinity_score',
 'orig_destination_distance',
 'random_bool',
 'comp1_rate',
 'comp1_inv',
 'comp1_rate_percent_diff',
 'comp2_rate',
 'comp2_inv',
 'comp2_rate_percent_diff',
 'comp3_rate',
 'comp3_inv',
 'comp3_rate_percent_diff',
 'comp4_rate',
 'comp4_inv',
 'comp4_rate_percent_diff',
 'comp5_rate',
 'comp5_inv',
 'comp5_rate_percent_diff',
 'comp6_rate',
 'comp6_inv',
 'comp6_rate_percent_diff',
 'comp7_rate',
 'comp7_inv',
 'comp7_rate_percent_diff',
 'comp8_rate',


In [7]:
print(len(train['srch_id'].unique()))

199795


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   date_time                    object 
 2   site_id                      int64  
 3   visitor_location_country_id  int64  
 4   visitor_hist_starrating      float64
 5   visitor_hist_adr_usd         float64
 6   prop_country_id              int64  
 7   prop_id                      int64  
 8   prop_starrating              int64  
 9   prop_review_score            float64
 10  prop_brand_bool              int64  
 11  prop_location_score1         float64
 12  prop_location_score2         float64
 13  prop_log_historical_price    float64
 14  position                     int64  
 15  price_usd                    float64
 16  promotion_flag               int64  
 17  srch_destination_id          int64  
 18  srch_length_of_stay          int64  
 19  

In [9]:
train.isna().sum()/len(train) #perc of missing obs

srch_id                        0.000000
date_time                      0.000000
site_id                        0.000000
visitor_location_country_id    0.000000
visitor_hist_starrating        0.949204
visitor_hist_adr_usd           0.948977
prop_country_id                0.000000
prop_id                        0.000000
prop_starrating                0.000000
prop_review_score              0.001485
prop_brand_bool                0.000000
prop_location_score1           0.000000
prop_location_score2           0.219902
prop_log_historical_price      0.000000
position                       0.000000
price_usd                      0.000000
promotion_flag                 0.000000
srch_destination_id            0.000000
srch_length_of_stay            0.000000
srch_booking_window            0.000000
srch_adults_count              0.000000
srch_children_count            0.000000
srch_room_count                0.000000
srch_saturday_night_bool       0.000000
srch_query_affinity_score      0.935986


In [10]:
train['srch_id'].value_counts().sort_index()

1         28
4         32
6          5
8         21
11        33
          ..
332777    32
332781    15
332782    24
332784    28
332785     6
Name: srch_id, Length: 199795, dtype: int64

## 1. Get smaller subset, computational limitations

In [11]:
train['srch_id'].unique()[:10000] #ids are not 1,2,3,...

array([    1,     4,     6, ..., 16672, 16673, 16674], dtype=int64)

In [18]:
#Smaller subset of 500 searches:
train_subset = train[train['srch_id'] < 864]
len(train_subset['srch_id'].unique())

500

In [19]:
d = {}
for search in list(train_subset['srch_id'].unique()):
    d["srch_id_{0}".format(search)] = train_subset[train_subset['srch_id']==search]

In [20]:
y = train_subset['booking_bool']
y.value_counts() #343 out of 500 searches have resulted in a booking

0    11755
1      343
Name: booking_bool, dtype: int64

In [28]:
train_subset.groupby('booking_bool').mean()

NameError: name 'train_subset' is not defined

## Competitive info

In [29]:
for comp in [1,2,3,4,5,6,7,8]:
    print("comp{0}_inv".format(comp))
    
    print(train["comp{0}_rate".format(comp)].isnull().sum())
    print(train["comp{0}_rate".format(comp)].value_counts())
    
    print(train["comp{0}_inv".format(comp)].isnull().sum())
    print(train["comp{0}_inv".format(comp)].value_counts())

comp1_inv
4838417
 1.0    67256
 0.0    42959
-1.0     9715
Name: comp1_rate, dtype: int64
4828788
 0.0    122599
 1.0      5492
-1.0      1468
Name: comp1_inv, dtype: int64
comp2_inv
2933675
 0.0    1613590
 1.0     250276
-1.0     160806
Name: comp2_rate, dtype: int64
2828078
 0.0    2010094
 1.0     101958
-1.0      18217
Name: comp2_inv, dtype: int64
comp3_inv
3424059
 0.0    1212580
 1.0     174118
-1.0     147590
Name: comp3_rate, dtype: int64
3307357
 0.0    1501978
 1.0     122194
-1.0      26818
Name: comp3_inv, dtype: int64
comp4_inv
4650969
 0.0    204349
-1.0     54057
 1.0     48972
Name: comp4_rate, dtype: int64
4614684
 0.0    301385
 1.0     37747
-1.0      4531
Name: comp4_inv, dtype: int64
comp5_inv
2735974
 0.0    1616232
 1.0     328268
-1.0     277873
Name: comp5_rate, dtype: int64
2598327
 0.0    2184479
 1.0     148457
-1.0      27084
Name: comp5_inv, dtype: int64
comp6_inv
4718190
 0.0    160932
 1.0     55022
-1.0     24203
Name: comp6_rate, dtype: int64
469737